### Bootstrapping data prep pipeline

This is a weakly supervised technique and should be augmented with human annotaions at a higher level. 


### Sampler function for data prep for training


- The implentation for the blocks in blue ink.
- A set of functions that takes as input :
    - CSV's at multiple levels - Image, Species count level or Image, bounding box level, etc.  
    - Sampling technique
    - number of samples required
    - this list will evolve as I write this code
    

<img src="IMG_0026.jpg" width="600" height="200">



1. Read all the shards created - ```/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/round1/bootstrap_data_<>```
2. Consolidate the list of filenames and animals
3. Sample them on the proportions that I build
4. Choose all the multi-species images

In [2]:
import pandas as pd
import csv, glob, random, os
from timeit import default_timer as timer
import numpy as np
import matplotlib.pyplot as plt

In [1]:
# add in parameters
round_number = 'round1_ckpt69k' # This is the round for which model has to be run with this sample
round_number_next = 'round2' # This is the round for which data is being prepared
Lower_Bound_experiment_path = 'bootstrap_LowerBound50p'
pred_files_path = '/home/ubuntu/data/tensorflow/my_workspace/training_demo/Predictions/S1_S6_lb50p/'

In [4]:
data_already_used_for_training_set = set()
list_used_images = glob.glob('../data/bootstrapping/{0}/*_csv/*.csv'.format(Lower_Bound_experiment_path))
for lst in list_used_images:
    data_already_used_inter = set(pd.read_csv(lst)['filename'])
    data_already_used_for_training_set = data_already_used_for_training_set.union(data_already_used_inter)
    print("File Name: {0}, Data Added: {1}".format(lst, len(data_already_used_inter)))

data_already_used_for_training_list = list(data_already_used_for_training_set)

print("Total data added so far: ", len(data_already_used_for_training_list))

File Name: ../data/bootstrapping/bootstrap_LowerBound50p/round1_csv/singleSpecies_bbox.csv, Data Added: 100000
Total data added so far:  100000


### Dataframe with all shards appended
Generate a master dataset with all the appended bounding boxes

In [5]:
df_base_bbox = pd.DataFrame()
start = timer()
for i, path in enumerate(glob.glob(os.path.join(pred_files_path, \
                                                  '{0}/Post_procession_of_infer_detection/bootstrap_data_snapshot_serengeti_s01_s06_*'.format(round_number)))):
    df = pd.read_csv(path)
    df_base_bbox = df_base_bbox.append(df)
end = timer()
print(round((end - start), 1))
print(df_base_bbox.shape)

7.4
(717868, 6)


In [6]:
df_base_bbox.head()

,filename,class,xmin,ymin,xmax,ymax
0,S3/B07/B07_R11/S3_B07_R11_IMAG0246,gazelleThomsons,0.890966,0.585981,0.997992,0.919471
1,S2/K09/K09_R1/S2_K09_R1_PICT0812,gazelleThomsons,0.693507,0.279107,0.982956,0.777959
2,S3/P09/P09_R12/S3_P09_R12_IMAG0382,wildebeest,0.135006,0.371933,0.573553,0.816625
3,S2/E13/E13_R1/S2_E13_R1_IMAG0147,warthog,0.376121,0.484358,0.485064,0.557452
4,S3/S09/S09_R12/S3_S09_R12_IMAG1322,zebra,0.386718,0.428981,0.494476,0.555750


In [7]:
# Exclude data that have already been used in training
# index_not_used_for_training = [filename in data_already_used_for_training_list for filename in df_base_bbox['filename']]
# index_not_used_for_training

df_base_bbox = df_base_bbox[~df_base_bbox['filename'].isin(data_already_used_for_training_list)]
print(df_base_bbox.shape)
df_base_bbox.head()

(582465, 6)


,filename,class,xmin,ymin,xmax,ymax
0,S3/B07/B07_R11/S3_B07_R11_IMAG0246,gazelleThomsons,0.890966,0.585981,0.997992,0.919471
1,S2/K09/K09_R1/S2_K09_R1_PICT0812,gazelleThomsons,0.693507,0.279107,0.982956,0.777959
2,S3/P09/P09_R12/S3_P09_R12_IMAG0382,wildebeest,0.135006,0.371933,0.573553,0.816625
3,S2/E13/E13_R1/S2_E13_R1_IMAG0147,warthog,0.376121,0.484358,0.485064,0.557452
4,S3/S09/S09_R12/S3_S09_R12_IMAG1322,zebra,0.386718,0.428981,0.494476,0.555750


### 1. HERD
We first focus on and sample the herd images. Since, the images are herd images are less than 5000 (2747 images to be precise), we take all of them for our model training.

In [8]:
df_image_popultion_dist = df_base_bbox.groupby(by=['filename'], as_index=False)['xmin'].count()
df_image_popultion_dist.columns = ['filename', 'count_box']
df_image_herd_lst = list(df_image_popultion_dist[df_image_popultion_dist['count_box'] >= 11]['filename'])
df_image_popultion_dist = df_image_popultion_dist.groupby(by=['count_box'], as_index=False)['filename'].count()
df_image_popultion_dist.columns = ['count_box', 'num_images']

# plt.bar(df_image_popultion_dist.count_box, df_image_popultion_dist.num_images)
df_image_popultion_dist.tail()

,count_box,num_images
16,17,44
17,18,28
18,19,6
19,20,5
20,21,1


In [9]:
len(df_image_herd_lst)

4346

## 2. Multi-Species images
1. Here I will sample 5000(or all) multi-species images. The training dataset does not contain multi-species images so I want to include multi-species images in further model trainig loop.
2. Exclude the images that were part of the **part 1 - HERD**

In [10]:
# 1. get unique filename and class. i.e, for each of the images gets the species in the data
# 2. Exclude the images that were part of HERD list
# 3. Get a dataframe that has image name and the animals in the image. 
start = timer() # time the process
df_base = df_base_bbox[['filename', 'class']]
df_base = df_base.drop_duplicates()
df_base = df_base[~df_base['filename'].isin(df_image_herd_lst)] # 2. Excluding images in HERD list
end = timer()
print("execution time: {0} Seconds".format(round(end - start, 1)))
print(df_base.shape)

execution time: 0.2 Seconds
(328373, 2)


In [11]:
# 1. From the above dataframe, get the list of images that have multiple animals.
# 2. Get the dataframe with image names and the species in it for the multi-species images

df_base_temp = df_base.groupby(by=['filename'], as_index=False)['class'].count() # groupby the get the distinct animals per image
df_base_temp.columns = ['filename', 'count_species'] # rename the columns

# Filtering the multispecies images
df_base_multi_species = df_base_temp[df_base_temp['count_species']>=2] # 1.
# Capturing the multi-species images in a list object 
df_base_multi_species_lst = list(set(df_base_multi_species.filename)) # 1.
print("# of images: {0}".format(len(set(df_base_multi_species['filename']))))
print(len(df_base_multi_species_lst))
df_base_multi_species.head()

# of images: 5089
5089


,filename,count_species
249,S1/B05/B05_R3/S1_B05_R3_PICT0500,2
250,S1/B05/B05_R3/S1_B05_R3_PICT0501,2
529,S1/B07/B07_R1/S1_B07_R1_PICT0459,2
530,S1/B07/B07_R1/S1_B07_R1_PICT0460,2
2363,S1/C05/C05_R3/S1_C05_R3_PICT2414,2


In [12]:
# Distribution of the image count for n-distinct species
species_dist = df_base_temp.groupby(by=['count_species'], as_index=False)['filename'].count()
species_dist

,count_species,filename
0,1,318181
1,2,5075
2,3,14


## 3. Single species images

In [16]:
# 1. Filter out the image, class that are single species
## a. df_base already excludes HERD images **(Part 1, HERD)**

df_single_species = df_base[~df_base['filename'].isin(df_base_multi_species_lst)]
print(df_single_species.shape)
df_single_species.head()

(318181, 2)


,filename,class
0,S3/B07/B07_R11/S3_B07_R11_IMAG0246,gazelleThomsons
1,S2/K09/K09_R1/S2_K09_R1_PICT0812,gazelleThomsons
2,S3/P09/P09_R12/S3_P09_R12_IMAG0382,wildebeest
3,S2/E13/E13_R1/S2_E13_R1_IMAG0147,warthog
4,S3/S09/S09_R12/S3_S09_R12_IMAG1322,zebra


#### Function to sample out the images given parameters like:
1. Species to sample
2. Number of images to sample
    - Sampling size is proportional to in the inverse frequency of the training data size of the species.
    - Considered species with less than 1000 images in the initial dataset.

In [17]:
def get_image_samples_for_species(dataframe, species, count, seed = None):
    '''This function takes a dataframe and samples out n(count)-records
    for the given species'''
    random.seed(seed)
    dataframe_species = dataframe[dataframe['class']==species]
    row, col = dataframe_species.shape
    if row < count:
        df_sampled_species = dataframe_species
    else:
        filename_sampled = random.sample(list(set(dataframe_species['filename'])), count)
        df_sampled_species = dataframe_species[[val in filename_sampled for val in list(dataframe_species['filename'])]]
    return df_sampled_species

def get_imget_image_samples_for_species_with_uniform_dist(dataframe, num_sample, seed=None):
    """Sample images with uniform and equal probability assumption for all species. """
    random.seed(seed)
    row, col = dataframe.shape
    filename_sampled = random.sample(list(set(dataframe['filename'])), num_sample)
    df_sampled = dataframe.loc[dataframe['filename'].isin(filename_sampled)]
    return df_sampled

In [18]:
# Test the function 
get_image_samples_for_species(df_single_species, 'wildebeest', 1)

,filename,class
126,S3/H04/H04_R11/S3_H04_R11_IMAG0117,wildebeest


In [19]:
pwd

'/home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/data_prep'

In [12]:
# 2(a) Import the dataset with sampling counts for Species
df_sample_size = pd.read_csv('../data/bootstrapping/{1}/sample_proportion_{0}.csv'\
                             .format(round_number_next, Lower_Bound_experiment_path), sep='\t')
df_sample_size.head()

,Species,Training_boxes_Count,image_count,inv_image_count,p_samples,num_sampled_images
0,ostrich,"1,007",676,0.00148,1.30%,65
1,topi,"1,690",674,0.00148,1.30%,66
2,eland,"1,812",665,0.00150,1.32%,66
3,human,871,629,0.00159,1.39%,70
4,impala,"2,317",622,0.00161,1.41%,71


In [16]:
# Initiate an empty dataset that will eventually contain the samples species and the corresponfing filename
df_all = pd.DataFrame()
for i in range(df_sample_size.shape[0]):
    species = df_sample_size.iloc[i]['Species'] # get the respective species to sample
    count = df_sample_size.iloc[i]['num_sampled_images'] # get count to sample
    df_sampled_species = get_image_samples_for_species(df_single_species, species, count)
    df_all = df_all.append(df_sampled_species)
df_sampled_species_lst = list(set(df_all['filename']))

In [17]:
print(sum(df_all.groupby(['class'], as_index=False)['filename'].count()['filename']))
print(len(df_sampled_species_lst))
df_all.tail()

819
819


,filename,class
3144,S5/D04/D04_R3/S5_D04_R3_IMAG0026,hippopotamus
2700,S5/D04/D04_R1/S5_D04_R1_IMAG1673,hippopotamus
3585,S5/D04/D04_R3/S5_D04_R3_IMAG0169,hippopotamus
576,S1/E03/E03_R2/S1_E03_R2_PICT0059,hippopotamus
4460,S6/F06/F06_R2/S6_F06_R2_IMAG0334,hippopotamus


## Export the datasets to CSV:
1. herd dataset
2. Multi-Species
3. Single Species

**1. HERD images**

In [19]:
df_herd = df_base_bbox[df_base_bbox['filename'].isin(df_image_herd_lst)]
df_herd.to_csv('../data/bootstrapping/{1}/{0}_csv/herd_bbox.csv'\
               .format(round_number_next, Lower_Bound_experiment_path), index=False)
print(df_herd.shape)

(21122, 6)


**2. Multi-Species**

In [20]:
df_multiSpecies = df_base_bbox[df_base_bbox['filename'].isin(df_base_multi_species_lst)]
df_multiSpecies.to_csv('../data/bootstrapping/{1}/{0}_csv/multiSpecies_bbox.csv'\
                       .format(round_number_next, Lower_Bound_experiment_path), index=False)
print(df_multiSpecies.shape)

(10236, 6)


**3. Single Species**

In [21]:
df_singleSpecies = df_base_bbox[df_base_bbox['filename'].isin(df_sampled_species_lst)]
df_singleSpecies.to_csv('../data/bootstrapping/{1}/{0}_csv/singleSpecies_bbox.csv'\
                        .format(round_number_next, Lower_Bound_experiment_path), index=False)
print(df_singleSpecies.shape)

(1088, 6)


### 4. Single Species with Uniform distribution sampling

In [20]:
df_single_species_uniform_dist_lst = list(get_imget_image_samples_for_species_with_uniform_dist(df_single_species, \
                                                                                       num_sample=100000)['filename'])
df_single_species_uniform_dist = df_base_bbox[df_base_bbox['filename'].isin(df_single_species_uniform_dist_lst)]
print(len(df_single_species_uniform_dist_lst))

df_single_species_uniform_dist.to_csv('../data/bootstrapping/{1}/{0}_csv/singleSpecies_bbox.csv'\
                                      .format(round_number_next, Lower_Bound_experiment_path), index=False)
df_single_species_uniform_dist.head()

100000


,filename,class,xmin,ymin,xmax,ymax
7,S5/H07/H07_R1/S5_H07_R1_IMAG2673,guineaFowl,0.614661,0.670146,0.845363,0.974787
10,S2/B07/B07_R2/S2_B07_R2_PICT2496,wildebeest,0.019187,0.000000,0.936146,0.945290
11,S2/T09/T09_R2/S2_T09_R2_PICT0407,zebra,0.331471,0.458952,0.558414,0.696997
12,S3/S10/S10_R12/S3_S10_R12_IMAG1007,wildebeest,0.928326,0.533921,0.999058,0.785036
13,S3/S10/S10_R12/S3_S10_R12_IMAG1007,wildebeest,0.064094,0.570880,0.096490,0.608668


# Species Distribution in the samples:

In [3]:
round_number = round_number_next
filepath_list = glob.glob('../data/bootstrapping/{1}/{0}_csv/*'\
                          .format(round_number, Lower_Bound_experiment_path))

In [4]:
filepath_list

['../data/bootstrapping/bootstrap_LowerBound50p/round2_csv/singleSpecies_bbox.csv']

In [5]:
df_pred_gt_consolidated = pd.DataFrame()
for filepath in filepath_list:
    # import csv to pandas
    df_pred_gt_temp = pd.read_csv(filepath)
    # subset the 2 columns
    df_pred_gt_temp = df_pred_gt_temp[['filename', 'class']]
    # Drop duplicates. We need to get the frequency of images for each animal
    df_pred_gt_temp = df_pred_gt_temp.drop_duplicates()
    df_pred_gt_consolidated = df_pred_gt_consolidated.append(df_pred_gt_temp)

# Get the frequency of images for each animal
df_bootstrap = df_pred_gt_consolidated.groupby(by=['class'], as_index=False)['filename'].count()
df_bootstrap.columns = ['species', 'Additional_Data_Size']

In [6]:
df_bootstrap.to_csv('/home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/EDA_and_ModelEvaluation/{1}/BootStrap_{0}_addition.csv'\
                    .format(round_number, Lower_Bound_experiment_path), index=False)